## Forked from belows:
* [[Inference] EfficientDet](https://www.kaggle.com/shonenkov/inference-efficientdet)
* [[WBF over TTA][Single Model] EfficientDet](https://www.kaggle.com/shonenkov/bayesian-optimization-wbf-efficientdet)

## Licenses
| Repository | License |
| :--- | :--- |
| efficientdet-pytorch | Apache License 2.0 |
| weighted-boxes-fusion | MIT License |
| timm | Apache License 2.0 |
| omegaconf | BSD 3-Clause "New" or "Revised" License |

In [1]:
!pip install --no-deps '../input/packageefficientnet/timm-0.1.30-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/packageefficientnet/pycocotools-2.0.1-cp37-cp37m-linux_x86_64.whl' > /dev/null
!pip install --no-deps '../input/packageefficientnet/omegaconf-2.0.0-py3-none-any.whl' > /dev/null

In [2]:
#import apex.amp as amp

In [3]:
import sys
sys.path.insert(0, '../input/efficientdetrwightman')
sys.path.insert(0, '../input/weightedboxesfusion')

from ensemble_boxes import *
import os
from datetime import datetime
import time
import random
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
from effdet.efficientdet import HeadNet

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [4]:
# Global Parameters
IMG_SIZE_ORG = 1024
img_size_global = 1024
batch_global = 1
epoch_global = 5
folder_global = 'weights'
weight_global = '../input/efficientdetweights/1024-fold0-50.bin'
test_test = True

# Import test data

In [5]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=img_size_global, width=img_size_global, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [6]:
if test_test:
    DATA_ROOT_PATH = '../input/global-wheat-detection/test'
else:
    DATA_ROOT_PATH = '../input/global-wheat-detection/train'

class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [7]:
dataset = DatasetRetriever(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]),
    transforms=get_valid_transforms()
)

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

# Load network

In [8]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size=img_size_global
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchPredict(net, config)
    net.eval();
    return net.cuda()

net = load_net(weight_global)

# Prepare test time augmentation

In [9]:
class BaseWheatTTA:
    """ author: @shonenkov """
    image_size = img_size_global

    def augment(self, image):
        raise NotImplementedError
    
    def batch_augment(self, images):
        raise NotImplementedError
    
    def deaugment_boxes(self, boxes):
        raise NotImplementedError

class TTAHorizontalFlip(BaseWheatTTA):
    """ author: @shonenkov """

    def augment(self, image):
        return image.flip(1)
    
    def batch_augment(self, images):
        return images.flip(2)
    
    def deaugment_boxes(self, boxes, scores):
        boxes[:, [1,3]] = self.image_size - boxes[:, [3,1]]
        return boxes, scores

class TTAVerticalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return image.flip(2)
    
    def batch_augment(self, images):
        return images.flip(3)
    
    def deaugment_boxes(self, boxes, scores):
        boxes[:, [0,2]] = self.image_size - boxes[:, [2,0]]
        return boxes, scores
    
class TTARotate90(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 1, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 1, (2, 3))
    
    def deaugment_boxes(self, boxes, scores):
        res_boxes = boxes.copy()
        res_boxes[:, [0,2]] = self.image_size - boxes[:, [1,3]]
        res_boxes[:, [1,3]] = boxes[:, [2,0]]
        return res_boxes, scores
    
class TTAColorFlip(BaseWheatTTA):
    def augment(self, image):
        return image.flip(0)
    
    def batch_augment(self, images):
        return images.flip(1)
    
    def deaugment_boxes(self, boxes, scores):
        return boxes, scores
    
class TTACrop(BaseWheatTTA):
    def __init__(self, size, loc, boundary=0.5):
        self.size = size
        self.pad = (img_size_global - self.size) // 2
        self.size = img_size_global - (2*self.pad)
        self.boundary = boundary
        self.loc = loc
    
    def augment(self, image):
        if self.loc == 0: # center
            image = image[:, self.pad:self.pad+self.size, self.pad:self.pad+self.size]
        elif self.loc == 1: # top-left
            image = image[:, 0:self.size, 0:self.size]
        elif self.loc == 2: # top-right
            image = image[:, 0:self.size, (self.pad*2):(self.pad*2+self.size)]
        elif self.loc == 3: # bottom-left
            image = image[:, (self.pad*2):(self.pad*2+self.size), 0:self.size]
        else: # bottom-right
            image = image[:, (self.pad*2):(self.pad*2+self.size), (self.pad*2):(self.pad*2+self.size)]
        
        image = cv2.resize(image.permute(1,2,0).cpu().numpy(), (img_size_global,img_size_global), interpolation=cv2.INTER_LINEAR)
        return torch.from_numpy(image).permute(2,0,1)
    
    def batch_augment(self, images):
        images = torch.stack([self.augment(image) for image in images])
        return images.cuda()
    
    def deaugment_boxes(self, boxes, scores):
        if scores is None:
            scores = np.random.rand(boxes.shape[0])
        
        box_score = np.concatenate((boxes, np.expand_dims(scores, axis=1)), axis=1)
        if self.loc == 0 or self.loc == 2 or self.loc == 4: # min x1
            box_score = box_score[np.min(box_score[:,0:1], axis=1) > 0+self.boundary]
        if self.loc == 0 or self.loc == 3 or self.loc == 4: # min y1
            box_score = box_score[np.min(box_score[:,1:2], axis=1) > 0+self.boundary]
        if self.loc == 0 or self.loc == 1 or self.loc == 3: # max x2
            box_score = box_score[np.max(box_score[:,2:3], axis=1) < img_size_global-1-self.boundary]
        if self.loc == 0 or self.loc == 1 or self.loc == 2: # max y2
            box_score = box_score[np.max(box_score[:,3:4], axis=1) < img_size_global-1-self.boundary]
        
        #box_score = box_score[np.min(box_score[:,0:4], axis=1) > 0+self.boundary]
        #box_score = box_score[np.max(box_score[:,0:4], axis=1) < img_size_global-1-self.boundary]
        
        boxes, scores = box_score[:,0:4], box_score[:,4]
        boxes = (boxes*(self.size/img_size_global))# + self.pad
        
        if self.loc == 0:
            boxes += self.pad
        else:
            if self.loc == 2 or self.loc == 4:
                boxes[:,0] += (2*self.pad)
                boxes[:,2] += (2*self.pad)
            if self.loc == 3 or self.loc == 4:
                boxes[:,1] += (2*self.pad)
                boxes[:,3] += (2*self.pad)
        return boxes, scores
    
class TTACenterReduce(BaseWheatTTA):
    def __init__(self, size):
        self.size = size
        self.pad = (img_size_global - self.size) // 2
        self.size = img_size_global - (2*self.pad)
        #self.boundary = 0.5
    
    def augment(self, image):
        image_out = torch.zeros(image.shape)
        image = cv2.resize(image.permute(1,2,0).cpu().numpy(), (self.size,self.size), interpolation=cv2.INTER_LINEAR)
        image = torch.from_numpy(image).permute(2,0,1)        
        image_out[:, self.pad:self.pad+self.size, self.pad:self.pad+self.size] = image
        return image_out
    
    def batch_augment(self, images):
        images = torch.stack([self.augment(image) for image in images])
        return images.cuda()
    
    def deaugment_boxes(self, boxes, scores):
        if scores is None:
            scores = np.random.rand(boxes.shape[0])
        boxes = ((boxes - self.pad)*img_size_global/self.size).clip(min=0, max=img_size_global)

        box_score = np.concatenate((boxes, np.expand_dims(scores, axis=1)), axis=1)
        box_score = box_score[np.min(box_score[:,2:4]-box_score[:,0:2], axis=1) > 0]
        box_score = box_score[np.max(box_score[:,2:4]-box_score[:,0:2], axis=1) <= img_size_global]
        
        boxes, scores = box_score[:,0:4], box_score[:,4]        
        return boxes, scores

class TTACompose(BaseWheatTTA):
    """ author: @shonenkov """
    def __init__(self, transforms):
        self.transforms = transforms
        
    def augment(self, image):
        for transform in self.transforms:
            image = transform.augment(image)
        return image
    
    def batch_augment(self, images):
        for transform in self.transforms:
            images = transform.batch_augment(images)
        return images
    
    def prepare_boxes(self, boxes):
        result_boxes = boxes.copy()
        result_boxes[:,0] = np.min(boxes[:, [0,2]], axis=1)
        result_boxes[:,2] = np.max(boxes[:, [0,2]], axis=1)
        result_boxes[:,1] = np.min(boxes[:, [1,3]], axis=1)
        result_boxes[:,3] = np.max(boxes[:, [1,3]], axis=1)
        return result_boxes
    
    def deaugment_boxes(self, boxes, scores=None):
        for transform in self.transforms[::-1]:
            boxes, scores = transform.deaugment_boxes(boxes, scores)
        return self.prepare_boxes(boxes), scores

In [10]:
def process_det(index, det, score_threshold=0.25):
    boxes = det[index].detach().cpu().numpy()[:,:4]    
    scores = det[index].detach().cpu().numpy()[:,4]
    boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
    boxes = (boxes).clip(min=0, max=img_size_global).astype(int)
    indexes = np.where(scores>score_threshold)
    boxes = boxes[indexes]
    scores = scores[indexes]

    return boxes, scores

# Test codes for TTA

In [11]:
"""
transform = TTACompose([
    #TTARotate90(),
    #TTAVerticalFlip(),
    #TTAResize(),
])
transform2 = TTACompose([
    #TTAHorizontalFlip(),
    #TTAVerticalFlip(),
    #TTARotate90(),
    TTACrop(size=800, loc=4, boundary=0.5),
])

fig, ax = plt.subplots(1, 4, figsize=(20, 8))

# original
image, image_id = dataset[1]
numpy_image = image.permute(1,2,0).cpu().numpy().copy()
ax[0].imshow(numpy_image);
ax[0].set_title('original')

# tta
tta_image = transform.augment(image)
print(tta_image.shape)
tta_image_numpy = tta_image.permute(1,2,0).cpu().numpy().copy()
tta_image_size = torch.tensor([[tta_image.shape[1], tta_image.shape[2]]])

det = net(tta_image.unsqueeze(0).float().cuda(), torch.tensor([1]).float().cuda(), img_size=tta_image_size.float().cuda())
boxes, scores = process_det(0, det)
del det
gc.collect()
for box in boxes:
    cv2.rectangle(tta_image_numpy, (box[0].astype(int), box[1].astype(int)),
                                   (box[2].astype(int)-1, box[3].astype(int)-1), (1, 0, 0), 2)
ax[1].imshow(tta_image_numpy);
ax[1].set_title('original with bbox')

# tta2
tta_image = transform2.augment(image)
print(tta_image.shape)
tta_image_numpy = tta_image.permute(1,2,0).cpu().numpy().copy()
tta_image_size = torch.tensor([[tta_image.shape[1], tta_image.shape[2]]])

det = net(tta_image.unsqueeze(0).float().cuda(), torch.tensor([1]).float().cuda(), img_size=tta_image_size.float().cuda())
boxes, scores = process_det(0, det)
del det
gc.collect()
for box in boxes:
    cv2.rectangle(tta_image_numpy, (box[0].astype(int), box[1].astype(int)),
                                   (box[2].astype(int)-1,  box[3].astype(int)-1), (1, 0, 0), 2)
ax[2].imshow(tta_image_numpy);
ax[2].set_title('TTA2')

# deaugment
boxes, _ = transform2.deaugment_boxes(boxes)
for box in boxes:
    cv2.rectangle(numpy_image, (box[0].astype(int), box[1].astype(int)),
                               (box[2].astype(int)-1, box[3].astype(int)-1), (1, 0, 0), 2)
ax[3].imshow(numpy_image);
ax[3].set_title('deaugment TTA2');
""";

# Make TTA combination

In [12]:
from itertools import product

tta_transforms = []
for tta_combination in product([TTAHorizontalFlip(), None], 
                               [TTAVerticalFlip(), None],
                               [TTARotate90(), None]):
                               #[TTACrop(size=400, loc=1), TTACrop(size=400, loc=2), TTACrop(size=400, loc=3), TTACrop(size=400, loc=4), None]):
    print([tta_transform for tta_transform in tta_combination if tta_transform])
    tta_transforms.append(TTACompose([tta_transform for tta_transform in tta_combination if tta_transform]))

[<__main__.TTAHorizontalFlip object at 0x7f57c79ad710>, <__main__.TTAVerticalFlip object at 0x7f57c50d5f10>, <__main__.TTARotate90 object at 0x7f57c50d5e50>]
[<__main__.TTAHorizontalFlip object at 0x7f57c79ad710>, <__main__.TTAVerticalFlip object at 0x7f57c50d5f10>]
[<__main__.TTAHorizontalFlip object at 0x7f57c79ad710>, <__main__.TTARotate90 object at 0x7f57c50d5e50>]
[<__main__.TTAVerticalFlip object at 0x7f57c50d5f10>, <__main__.TTARotate90 object at 0x7f57c50d5e50>]
[]


# Make prediction with TTA

In [13]:
def make_tta_predictions(images, score_threshold=0.25):
    with torch.no_grad():
        images = torch.stack(images).float().cuda()
        predictions = []
        for tta_transform in tta_transforms:
            result = []
            #images_batch = tta_transform.batch_augment(images.clone())
            img_sizes = torch.stack((torch.ones([2,])*images.shape[2],
                                     torch.ones([2,])*images.shape[3])).permute(1,0)
            img_sizes = img_sizes.int().cuda()
            det = net(tta_transform.batch_augment(images.clone()), torch.tensor([1]*images.shape[0]).float().cuda(), img_size=img_sizes)

            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]
                scores = det[i].detach().cpu().numpy()[:,4]
                indexes = np.where(scores > score_threshold)[0]
                boxes = boxes[indexes]
                scores = scores[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                boxes, scores = tta_transform.deaugment_boxes(boxes.copy(), scores.copy())
                result.append({
                    'boxes': boxes,
                    'scores': scores,
                })
            predictions.append(result)
    return predictions

def run_wbf(predictions, image_index, image_size=img_size_global, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    boxes = [(prediction[image_index]['boxes']/image_size).tolist() for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist() for prediction in predictions]
    
    #for prediction in predictions:
        #print(prediction[image_index]['scores'].shape)
    #print('-'*15)
    
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).astype(int).tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size)
    return boxes, scores, labels

# Test codes for prediction

In [14]:
"""
# Show sample result of TTA image
import matplotlib.pyplot as plt

sample_idx = 0 # 0-9
for j, (images, image_ids) in enumerate(data_loader):
    if sample_idx//2 == j:
        break
i = sample_idx%2

predictions = make_tta_predictions(images)
sample = images[i].permute(1,2,0).cpu().numpy()

boxes, scores, labels = run_wbf(predictions, image_index=i)
boxes = boxes.astype(np.int32).clip(min=0, max=img_size_global)

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample, (box[0], box[1]), (box[2]-1, box[3]-1), (1, 0, 0), 2)
    
ax.imshow(sample);
""";

# Predict and format for final submission

In [15]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [16]:
#results = []
results_1 = pd.DataFrame(data=None, columns=['image_id','x','y','w','h'])
bbox_count = 0

from tqdm import tqdm
for images, image_ids in tqdm(data_loader):
    predictions = make_tta_predictions(images)
    for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i)
        boxes = (boxes*IMG_SIZE_ORG/img_size_global).astype(np.int32).clip(min=0, max=IMG_SIZE_ORG)
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        result = pd.DataFrame(data=None, columns=['image_id','x','y','w','h'], index=range(bbox_count,bbox_count+boxes.shape[0]))
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'image_id'] = image_id
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'x'] = boxes[:,0]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'y'] = boxes[:,1]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'w'] = boxes[:,2]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'h'] = boxes[:,3]
        bbox_count += boxes.shape[0]
        """
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)
        """
        results_1 = pd.merge(results_1, result, how='outer')

100%|██████████| 5/5 [00:12<00:00,  2.55s/it]


In [17]:
class TrainGlobalConfig:
    num_workers = 4
    batch_size = batch_global
    n_epochs = epoch_global
    lr = 0.0002
    mixed_precision = True
    accumulate = 16

    resume = True
    resume_path = weight_global

    folder = folder_global

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#     SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#     scheduler_params = dict(
#         max_lr=0.001,
#         epochs=n_epochs,
#         steps_per_epoch=int(len(train_dataset) / batch_size),
#         pct_start=0.1,
#         anneal_strategy='cos', 
#         final_div_factor=10**5
#     )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=True, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

In [18]:
def get_train_transforms():
    return A.Compose(
        [
            A.Resize(height=img_size_global, width=img_size_global, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [19]:
TRAIN_ROOT_PATH = DATA_ROOT_PATH

class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
        # (1) Apply cutmix augmentation
        image, boxes = self.load_image_and_boxes(index)
        
        # (2) Apply albumentation
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes

In [20]:
train_dataset = DatasetRetriever(
    image_ids=np.unique(results_1['image_id'].values),
    #image_ids=results_1['image_id'].values,
    marking=results_1,
    transforms=get_train_transforms(),
    test=False,
)

In [21]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [22]:
import warnings
import shutil

warnings.filterwarnings("ignore")

class Fitter:
    
    def __init__(self, model, device, config):
        self.model = model
        self.device = device
        self.config = config

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        #self.model, self.optimizer = amp.initialize(self.model, self.optimizer, opt_level='O1', verbosity=0)

        if self.config.resume:
            self.load(self.config.resume_path)
        else:
            self.epoch = 0
            self.best_summary_loss = 10**5
            
        self.base_dir = f'./{config.folder}'
        if os.path.exists(self.base_dir):
            shutil.rmtree(self.base_dir)
        os.makedirs(self.base_dir)
        self.log_path = f'{self.base_dir}/log.txt'
            
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader):
        epoch_range = range(0, self.config.n_epochs)
        for e in epoch_range:
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')
            """
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-1]:
                    os.remove(path)
            """

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        #t = time.time()
        for step, (images, targets, image_ids) in enumerate(train_loader):
            """
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            """
            images = torch.stack(images)
            images = images.to(self.device).float()
            batch_size = images.shape[0]
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]
            
            self.optimizer.zero_grad()

            outputs = self.model(images, {'bbox':boxes, 'cls':labels})
            loss = outputs['loss']

            #with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                #scaled_loss.backward()
            loss.backward()

            summary_loss.update(loss.detach().item(), batch_size)
            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            #'amp_state_dict': amp.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        #amp.load_state_dict(checkpoint['amp_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        print('load completed')
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [23]:
def collate_fn(batch):
    return tuple(zip(*batch))

def run_training():
    device = torch.device('cuda:0')
    net.to(device)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=RandomSampler(train_dataset),
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
        collate_fn=collate_fn,
    )

    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    fitter.fit(train_loader)

In [24]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
def get_net():
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)
    checkpoint = torch.load('../input/efficientdetweights/tf_efficientdet_d5-ef44aea8.pth')
    net.load_state_dict(checkpoint)
    config.num_classes = 1
    config.image_size = img_size_global
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    return DetBenchTrain(net, config)

In [25]:
net = get_net()
run_training()

load completed
Fitter prepared. Device is cuda:0

2020-08-04T06:40:01.974242
LR: 1.5625e-06
[RESULT]: Train. Epoch: 38, summary_loss: 0.19584, time: 7.91803

2020-08-04T06:40:10.531170
LR: 1.5625e-06
[RESULT]: Train. Epoch: 39, summary_loss: 0.19089, time: 8.09142

2020-08-04T06:40:19.551874
LR: 1.5625e-06
[RESULT]: Train. Epoch: 40, summary_loss: 0.18560, time: 8.00396

2020-08-04T06:40:28.599920
LR: 1.5625e-06
[RESULT]: Train. Epoch: 41, summary_loss: 0.18084, time: 7.69959

2020-08-04T06:40:37.337584
LR: 1.5625e-06
[RESULT]: Train. Epoch: 42, summary_loss: 0.17694, time: 7.75099


In [26]:
net = load_net('./weights/last-checkpoint.bin')
#results = []
results_2 = pd.DataFrame(data=None, columns=['image_id','x','y','w','h'])
bbox_count = 0

from tqdm import tqdm
for images, image_ids in tqdm(data_loader):
    predictions = make_tta_predictions(images)
    for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i)
        boxes = (boxes*IMG_SIZE_ORG/img_size_global).astype(np.int32).clip(min=0, max=IMG_SIZE_ORG)
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        result = pd.DataFrame(data=None, columns=['image_id','x','y','w','h'], index=range(bbox_count,bbox_count+boxes.shape[0]))
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'image_id'] = image_id
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'x'] = boxes[:,0]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'y'] = boxes[:,1]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'w'] = boxes[:,2]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'h'] = boxes[:,3]
        bbox_count += boxes.shape[0]
        """
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)
        """
        results_2 = pd.merge(results_2, result, how='outer')

100%|██████████| 5/5 [00:11<00:00,  2.22s/it]


In [27]:
class TrainGlobalConfig:
    num_workers = 4
    batch_size = batch_global
    n_epochs = epoch_global
    lr = 0.0002
    mixed_precision = True
    accumulate = 16

    resume = True
    resume_path = './weights/last-checkpoint.bin'

    folder = folder_global

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#     SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#     scheduler_params = dict(
#         max_lr=0.001,
#         epochs=n_epochs,
#         steps_per_epoch=int(len(train_dataset) / batch_size),
#         pct_start=0.1,
#         anneal_strategy='cos', 
#         final_div_factor=10**5
#     )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=True, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

In [28]:
train_dataset = DatasetRetriever(
    image_ids=np.unique(results_2['image_id'].values),
    #image_ids=results_1['image_id'].values,
    marking=results_2,
    transforms=get_train_transforms(),
    test=False,
)

In [29]:
net = get_net()
run_training()

load completed
Fitter prepared. Device is cuda:0

2020-08-04T06:41:01.913991
LR: 1.5625e-06
[RESULT]: Train. Epoch: 43, summary_loss: 0.18357, time: 7.59892

2020-08-04T06:41:10.137663
LR: 1.5625e-06
[RESULT]: Train. Epoch: 44, summary_loss: 0.17881, time: 8.05535

2020-08-04T06:41:19.371089
LR: 1.5625e-06
[RESULT]: Train. Epoch: 45, summary_loss: 0.17370, time: 7.80852

2020-08-04T06:41:28.085892
LR: 1.5625e-06
[RESULT]: Train. Epoch: 46, summary_loss: 0.16874, time: 7.73465

2020-08-04T06:41:36.683501
LR: 1.5625e-06
[RESULT]: Train. Epoch: 47, summary_loss: 0.16704, time: 7.69386


In [30]:
net = load_net('./weights/last-checkpoint.bin')

results = []
#results_1 = pd.DataFrame(data=None, columns=['image_id','x','y','w','h'])
bbox_count = 0

from tqdm import tqdm
for images, image_ids in tqdm(data_loader):
    predictions = make_tta_predictions(images)
    for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i)
        boxes = (boxes*IMG_SIZE_ORG/img_size_global).astype(np.int32).clip(min=0, max=IMG_SIZE_ORG)
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        """
        result = pd.DataFrame(data=None, columns=['image_id','x','y','w','h'], index=range(bbox_count,bbox_count+boxes.shape[0]))
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'image_id'] = image_id
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'x'] = boxes[:,0]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'y'] = boxes[:,1]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'w'] = boxes[:,2]
        result.loc[bbox_count:bbox_count+boxes.shape[0], 'h'] = boxes[:,3]
        bbox_count += boxes.shape[0]
        """
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)
        
        #results_1 = pd.merge(results_1, result, how='outer')

100%|██████████| 5/5 [00:10<00:00,  2.20s/it]


In [31]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
print(test_df.shape)
test_df.head()

(10, 2)


,image_id,PredictionString
0,2fd875eaa,0.9641 888 49 107 95 0.9579 729 885 100 92 0.9...
1,cc3532ff6,0.9642 771 828 166 162 0.9234 909 123 113 98 0...
2,51b3e36ab,0.9324 874 290 149 141 0.9005 362 153 102 99 0...
3,53f253011,0.9248 20 597 119 143 0.9219 232 839 116 96 0....
4,cb8d261a3,0.8637 434 116 109 86 0.8608 23 862 81 151 0.8...
